[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jolin-io/fall-in-love-with-julia/main?filepath=04%20differential%20equations%20-%2002%20stochastic.ipynb)

# Stochastic Differential Equations (SDE)

This tutorial will introduce you to the functionality for solving SDEs.


# Example 1: Scalar SDEs

In this example we will solve the equation

$$du = f(u,p,t)dt + g(u,p,t)dW$$

where $f(u,p,t)=αu$ and $g(u,p,t)=βu$. We know via Stochastic Calculus that the solution to this equation is

$$u(t,Wₜ)=u₀\exp((α-\frac{β^2}{2})t+βWₜ)$$

To solve this numerically, we define a problem type by giving it the equation and the initial condition:

In [ ]:
using DifferentialEquations
α=1
β=1
u₀=1/2
f(u,p,t) = α*u
g(u,p,t) = β*u
dt = 1//2^(4)
tspan = (0.0,1.0)
prob = SDEProblem(f,g,u₀,(0.0,1.0))

The solve interface is then the same as with ODEs. Here we will use the classic Euler-Maruyama algorithm EM and plot the solution:

In [ ]:
using Plots; plotly() # Using the Plotly backend

sol = solve(prob,EM(),dt=dt)
plot(sol)

## Using Higher Order Methods

One unique feature of DifferentialEquations.jl is that higher-order methods for stochastic differential equations are included. For reference, let's also give the SDEProblem the analytical solution. We can do this by making a test problem. This can be a good way to judge how accurate the algorithms are, or is used to test convergence of the algorithms for methods developers. Thus we define the problem object with:

In [ ]:
f_analytic(u₀,p,t,W) = u₀*exp((α-(β^2)/2)*t+β*W)
ff = SDEFunction(f,g,analytic=f_analytic)
prob = SDEProblem(ff,g,u₀,(0.0,1.0))

and then we pass this information to the solver and plot:

In [ ]:
# We can plot using the classic Euler-Maruyama algorithm as follows:
sol = solve(prob,EM(),dt=dt)
plot(sol,plot_analytic=true)

We can choose a higher-order solver for a more accurate result, here `SRIW1`.

If `dt` is not given, the solver will automatically determine a starting ``dt``. This estimate at the beginning is conservative (small) to ensure accuracy. We can instead start the method with a larger ``dt`` by passing in a value for the starting ``dt``.

In [ ]:
sol = solve(prob, SRIW1(), dt=dt)
plot(sol, plot_analytic=true)

## Ensemble Simulations

Instead of solving single trajectories, we can turn our problem into a ``EnsembleProblem`` to solve many trajectories all at once. This is done by the ``EnsembleProblem`` constructor:

In [ ]:
ensembleprob = EnsembleProblem(prob)

The solver commands are defined [at the Parallel Ensemble Simulations page](https://diffeq.sciml.ai/stable/features/ensemble/#ensemble). For example we can choose to have 1000 trajectories via ``trajectories=1000``. In addition, this will automatically parallelize using Julia native parallelism if extra processes are added via ``addprocs()``, but we can change this to use multithreading via ``EnsembleThreads()``. Together, this looks like:

In [ ]:
sol = solve(ensembleprob,EnsembleThreads(),trajectories=1000)

Many more controls are defined at the [Ensemble simulations page](https://diffeq.sciml.ai/stable/features/ensemble/#ensemble), including analysis tools. A very simple analysis can be done with the ``EnsembleSummary``, which builds mean/var statistics and has an associated plot recipe. For example, we can get the statistics at every ``0.01`` timesteps and plot the average + error using:

In [ ]:
using DifferentialEquations.EnsembleAnalysis
summ = EnsembleSummary(sol,0:0.01:1)
plot(summ,labels="Middle 95%")
summ = EnsembleSummary(sol,0:0.01:1;quantiles=[0.25,0.75])
plot!(summ,labels="Middle 50%",legend=true)

Additionally we can easily calculate the correlation between the values at ``t=0.2`` and ``t=0.7`` via

In [ ]:
timepoint_meancor(sol, 0.2,0.7) # Gives both means and then the correlation coefficient

In [ ]:
timeseries_point_mean(sol, [0.2, 0.3, 0.5])

# Example 2: Systems of SDEs with Diagonal Noise

More generally, an SDE

$$du = f(u,p,t)dt + g(u,p,t)dW$$

generalizes to systems of equations is done in the same way as ODEs. Here, g is now a matrix of values. One common case, and the default for DifferentialEquations.jl, is diagonal noise where ``g`` is a diagonal matrix. This means that every function in the system gets a different random number. Instead of handling matrices in this case, we simply define both ``f`` and ``g`` as in-place functions. Thus ``f(du,u,p,t)`` gives a vector of ``du`` which is the deterministic change, and ``g(du2,u,p,t)`` gives a vector ``du2`` for which ``du2.*W`` is the stochastic portion of the equation.

For example, the Lorenz equation with additive noise has the same deterministic portion as the Lorenz equations, but adds an additive noise, which is simply ``3*N(0,dt)`` where ``N`` is the normal distribution dt is the time step, to each step of the equation. This is done via:

In [ ]:
function lorenz(du,u,p,t)
  du[1] = 10.0(u[2]-u[1])
  du[2] = u[1]*(28.0-u[3]) - u[2]
  du[3] = u[1]*u[2] - (8/3)*u[3]
end

function σ_lorenz(du,u,p,t)
  du[1] = 3.0
  du[2] = 3.0
  du[3] = 3.0
end

prob_sde_lorenz = SDEProblem(lorenz,σ_lorenz,[1.0,0.0,0.0],(0.0,10.0))
sol = solve(prob_sde_lorenz)
plot(sol,vars=(1,2,3))

Note that it's okay for the noise function to mix terms. For example

In [ ]:
function σ_lorenz(du,u,p,t)
  du[1] = sin(u[3])*3.0
  du[2] = u[2]*u[1]*3.0
  du[3] = 3.0
end

prob_sde_lorenz = SDEProblem(lorenz,σ_lorenz,[1.0,0.0,0.0],(0.0,10.0))
sol = solve(prob_sde_lorenz)
plot(sol,vars=(1,2,3))

# Example 3: Spatially-Colored Noise in the Heston Model

Colored noise can be defined using the [Noise Process interface](https://diffeq.sciml.ai/stable/features/noise_process/#noise_process). In that portion of the docs, it is shown how to define your own noise process ``my_noise``, which can be passed to the SDEProblem

```julia
SDEProblem(f,g,u0,tspan,noise=my_noise)
```

Note that general colored noise problems are only compatible with the ``EM`` and ``EulerHeun`` methods. This is discussed in the [SDE solvers page](https://diffeq.sciml.ai/stable/solvers/sde_solve/#sde_solve).

Let's define the Heston equation from financial mathematics:

$$dS = μSdt + \sqrt{v}SdW_1 \\ dv = κ(Θ-v)dt + σ\sqrt{v}dW_2 \\ dW_1 dW_2 = ρ dt$$

In this problem, we have a diagonal noise problem given by:

In [ ]:
function f(du,u,p,t)
  du[1] = μ*u[1]
  du[2] = κ*(Θ-u[2])
end
function g(du,u,p,t)
  du[1] = √u[2]*u[1]
  du[2] = Θ*√u[2]
end

However, our noise has a correlation matrix for some constant ``ρ``.

In [ ]:
ρ = 0.2
Γ = [1 ρ;ρ 1]

To solve this, we can define a ``CorrelatedWienerProcess`` which starts at zero ``(W(0)=0)`` via:

In [ ]:
heston_noise = CorrelatedWienerProcess!(Γ,tspan[1],zeros(2),zeros(2))

This is then used to build the SDE:
```julia
SDEProblem(f,g,u0,tspan,noise=heston_noise)
```
Of course, to fully define this problem we need to define our constants. Constructors for making common models like this easier to define can be found in the modeling toolkits. For example, the ``HestonProblem`` is pre-defined as part of the [financial modeling tools](https://diffeq.sciml.ai/stable/models/financial/#financial_models).